# Practical session 4: Modulating internal states and Sensing other robot's attributes

In the last practical session, we saw how to run multiple behaviors in parallel on multiple robots. In this section we will see more advanced methods for combining behaviors by modulating their activations according to internal states of the robot and by allowing them to sense the attributes of others. In order to start with a clean basis, let's first provide the definition of several behaviors. These four behaviors are simply implementations of the [Braitenberg vehicles](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0) we have seen in class, where the sources (i.e. what is sensed by the proximeters) are other robots:

## Comment 

- Make sure to untick and tick back the `hiden non existing` button in the simulator interface

In [1]:
def fear(robot):
    left, right = robot.sensors(sensed_entities=["robots"])
    left_wheel = left
    right_wheel = right
    return left_wheel, right_wheel

def aggression(robot):
    left, right = robot.sensors(sensed_entities=["robots"])
    left_wheel = right
    right_wheel = left
    return left_wheel, right_wheel

def love_cuddly(robot):
    left, right = robot.sensors(sensed_entities=["robots"])
    left_wheel = 1 - left
    right_wheel = 1 - right   
    return left_wheel, right_wheel

def love_shy(robot):
    left, right = robot.sensors(sensed_entities=["robots"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

Note that the arguments of the behavior functions above are now named `robot`, whereas they were named `robot`in the previous sessions. This is actually strictly equivalent, the argument name being an arbitrary name. We choose here to call it `robot` in order to avoid a confusion when dealing with multiple robots. The only important thing is that you use the same name in the function body (i.e `robot` here).

We define a fifth behavior for obstacle avoidance, where obstacles are walls, pilars, trees and cups:

In [2]:
def obstacle_avoidance(robot):
    left, right = robot.sensors(sensed_entities=["s_obstacles", "b_obstacles"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

Open V-REP and load the scene `robot-scene-4.ttt` located in the directory `robotics/pyvrep_robot/vrep-scenes`.

**Reminder:** First import the functions `open_session` and `close_session`. Then, obtain a reference to the simulator and the E-Puck by calling `open_session`. Since there are three robots in the scene, we call it like this:

In [3]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
start_server_and_interface(scene_name="session_4")

/home/cleger/Desktop/code/vivarium/vivarium/utils

STARTING SERVER


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[2024-12-02 18:45:16,291][__main__][INFO] - Scene running: session_4
[2024-12-02 18:45:18,836][vivarium.simulator.simulator][INFO] - Simulator initialized

STARTING INTERFACE


2024-12-02 18:45:21,678 Starting Bokeh server version 3.3.4 (running on Tornado 6.4)
2024-12-02 18:45:21,678 User authentication hooks NOT provided (default user enabled)
2024-12-02 18:45:21,679 Bokeh app running at: http://localhost:5006/run_interface
2024-12-02 18:45:21,679 Starting Bokeh server with process id: 70767


In [5]:
controller = NotebookController()

/home/cleger/Desktop/code/vivarium/venv/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


In [6]:
controller.run()

In [8]:
controller.print_fps()

measuring the FPS (number of steps per second) in the controller during 2 seconds ...


FPS: 31.00


Then execute the code you will encounter in the notebook or the code you will write for answering the questions. Whenever you want to restart from scratch (e.g. because something goes wrong or because you want to restart from scratch), first close the session by executing:

This will properly close all the running processes. Then restart the notebook (`Kernel -> Restart`) and open the session again:

And you will restart with a clean session.

## Emergent behaviors

When running one or several behaviors on a robot, you might observe behavioral responses that are not present in any of the behavior definitions. This usually doesn't mean that there is a bug, it could just be the result of an emergent behavioral property. Let's analyze this on a quick example, by running the `obstacle_avoidance` behavior on each robot:

In [12]:
for agent in controller.agents:
    agent.attach_behavior(obstacle_avoidance)
    agent.start_all_behaviors()

In [13]:
for agent in controller.agents:
    agent.print_behaviors()

Available behaviors: ['obstacle_avoidance'], Active behaviors: ['obstacle_avoidance']
Available behaviors: ['obstacle_avoidance'], Active behaviors: ['obstacle_avoidance']
Available behaviors: ['obstacle_avoidance'], Active behaviors: ['obstacle_avoidance']


In [15]:
# Update agents diet
for agent in controller.agents:
    agent.diet = ["resources"]

# Start resources apparition
controller.start_resources_apparition(interval=60)

# Start the eating mechanism
controller.start_eating_mechanism(interval=10)

Note that the cell above is a shortcut to execute the same set of instructions (the indented lines) on all the robots returned by the `open_session` function (the one you use to connect to the simulator). It can become very useful when you want to run the same set of instructions on all the robots, as it is the case here. 

**Q1:** You will observe that the robots seem to be attracted to objects that are not handled by the behavior (e.g. other robots). Analyse this phenomena and explain below why it is occuring in a few lines:

*Double click on this cell to enter your answer*

## Analyzing robot interactions

Let's analyze the interaction between two robots that run different behaviors. First close the current session and open a new one with only two robots that will be controlled (the third robot is still part of the scene but we don't need it here):

In [16]:
robot_0 = controller.agents[0]
robot_1 = controller.agents[1]
robot_2 = controller.agents[2]

# add a specific list for robots 0 and 1
robots_lst = [robot_0, robot_1]

In [17]:
for robot in robots_lst:
    print(f"setting attributes for robot {robot.idx}")
    robot.diameter = 7
    robot.color = "cyan" 

setting attributes for robot 0
setting attributes for robot 1


**Q2:** In addition to the `obstacle_avoidance` behavior, run several combinations of the four other behaviors defined above. For example, run `obstacle_avoidance` and `fear` on `robot1` ; and `obstacle_avoidance` and `aggression` on `robot2`. Find two of these combinations that you consider as interesting (e.g. because they result in a relatively complex interaction pattern, or because they can be linked to animal behavior) and describe them in a few lines:

In [18]:
# Your code here


*Your answer here*

## Weighting behaviors

As we have seen in the previous session, it is possible to run several behaviors in parallel on the same robot. When doing it, the motor activation sent to each wheel corresponds to the average of the motor activation returned by each behavior (this averaging is implemented internally, you don't need to worry about it). 

It is also possible to specify the weight of each running behavior, i.e. how much it will count in the averaging. This is done by returning three values in the function defining a behavior (instead of two as we were doing until now: one for the left wheel activation and one for the right one). For example, if we want to run the `obstacle_avoidance` behavior with a weight of 1 and the `fear` behavior with a weight of 0.5, we write:

In [19]:
print(controller.routine_handler._routines)

{'spawn_entity_routine': (functools.partial(<function spawn_entity_routine at 0x7ddf7e76d360>, entity_type=1, position_range=None), 60), 'eating_routine_range': (<function eating_routine_range at 0x7ddefc502830>, 10)}


In [20]:
controller.pause()

In [21]:
controller.run()

Simulator is already stopped


In [22]:
# Update agents diet
for agent in controller.agents:
    agent.diet = ["resources"]

# Start resources apparition
controller.start_resources_apparition(interval=60)

# Start the eating mechanism
controller.start_eating_mechanism(interval=10)

In [23]:
# First detach all behaviors on both robots:
for robot in robots_lst:
    robot.detach_all_behaviors()
    robot.stop_motors()
    
def obstacle_avoidance(robot):
    left, right = robot.sensors(sensed_entities=["s_obstacles", "b_obstacles"])
    return 1 - right, 1 - left

# Attach and start both behaviors on both robots:
for robot in robots_lst:
    print(f"\nattaching behaviors for robot {robot.idx}")
    robot.attach_behavior(obstacle_avoidance, weight=1)
    robot.attach_behavior(fear, weight=0.5)
    robot.start_all_behaviors()
    robot.print_behaviors()


attaching behaviors for robot 0
Available behaviors: ['obstacle_avoidance', 'fear'], Active behaviors: ['obstacle_avoidance', 'fear']

attaching behaviors for robot 1
Available behaviors: ['obstacle_avoidance', 'fear'], Active behaviors: ['obstacle_avoidance', 'fear']


By doing this, the wheel activations returned by the `obstacle_avoidance` behavior will have twice more weight than those returned by the `fear` behavior. For example, if `obstacle_avoidance` returns 0.6 for the left wheel, and `fear` returns 0.9, then the total activation of that wheel will be $(0.6 * 1 + 0.9 * 0.5) / (1 + 0.5) = 0.7$ (i.e. the average of both values weighted by their respective activation). Note that when no weight is provided in a behavior definition (i.e. when the behavior function returns only two values as usual), the corresponding behavior is set with a default weight of one.

## Weighting behaviors according to internal states

This weighting is particularly useful to activate a behavior according to some internal states of the robot. Let's consider a robot that eats spheres (as in the previous session) and that eating those spheres allows to raise a simulated energy level of the robot. We want to continuously compute the energy level of the robot according to how much spheres it has recently eaten. To do so, we first need a way to know when a robot has eaten for the last time. There are several ways of doing this that can serve in different use cases:

- agent.has_eaten(): function that tells if the agent has eaten since the last call to this function, defaults to False
- agent.time_since_feeding: attribute that tells the time since the last meal, defaults to infinity when the agent has never eaten
- agent.has_eaten_since(t): function that tells if the agent has eaten since time t

In [24]:
print(robot_2.has_eaten())
print(robot_2.time_since_feeding)
print(robot_2.has_eaten_since(50))

False
inf
False


We can also do it for all robots at once :

In [31]:
for robot in controller.agents:
    print(f"robot {robot.idx}: -ate: {robot.has_eaten()} -time since feeding: {robot.time_since_feeding}")
    print(robot.has_eaten_since(503))

robot 0: -ate: False -time since feeding: 502
True
robot 1: -ate: False -time since feeding: 1872
False
robot 2: -ate: False -time since feeding: 1012
False


In [27]:
controller.pause()

The cell above return `True` if the robot (`robot1` here) has eaten a sphere since the last call of the function. Run both the `obstacle avoidance` and the `foraging` behavior on an robot and execute the `has_eaten` function above at different times to understand correctly how it works. Remember we have already defined a `foraging_behavior` in the last session, which can be implemented like this:

In [35]:
def foraging(robot):
    left, right = robot.sensors(sensed_entities=["resources"])
    left_activation = right
    right_activation = left
    return left_activation, right_activation

In [36]:
controller.print_fps()

measuring the FPS (number of steps per second) in the controller during 2 seconds


FPS: 26.00


In [37]:
# First start sphere apparition faster in the environment:
controller.stop_resources_apparition()
controller.start_resources_apparition(interval=100)

In [38]:
# Detach all existing behaviors, and attach then start obstacle_avoidance and foraging on all robots:
for robot in controller.agents:
    robot.detach_all_behaviors()
    robot.attach_behavior(obstacle_avoidance, interval=10)
    robot.attach_behavior(foraging, interval=10)
    robot.start_all_behaviors()

Now we want to continuously compute the energy level of a robot, so that the level decreases slowly when nothing is eaten and increases whenever food is consumed. This can be done by attaching and starting a *routine* to a robot. The definition of a routine is very similar to the definition of a behavior, except that it doesn't return any value (whereas a behavior always returns the left and right wheel activations, and optionally a weight). Thus, a routine corresponds to a set of instructions that are executed at a particular frequency (as in a behavior), e.g. to compute some robot's internal states according to its interaction with the environment.

Let's define a routine called `foraging_drive` that computes the energy level as specified above:

## Comment : 

- Old values were really little and hard to keep energy level > 0 
- did that atm but will be changed (just for testing purposes)

In [39]:
max_energy_level = 100.
init_energy_level = 50.

In [40]:
for agent in controller.agents:
    agent.energy_level = init_energy_level
    print(f"Agent {agent.idx} energy level is: {agent.energy_level}")

Agent 0 energy level is: 50.0
Agent 1 energy level is: 50.0
Agent 2 energy level is: 50.0


In [41]:
def foraging_drive(robot): 
    if robot.has_eaten():
        # increase size and energy level
        robot.energy_level += 10  # if the robot has eaten a sphere, increase its energy level by 0.2
        robot.diameter += 0.5
    else:
        # decrease energy level
        robot.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level by 0.01
    # The line below bounds the value of the energy level between 0 and 1
    robot.energy_level = min(max_energy_level, max(robot.energy_level, 0.))

As for a behavior, the function defining a routine takes a `robot` as an argument representing the robot on which the routine is attached. Attaching a routine is done with: 

In [42]:
# assign foraging_drive to robot_0 and give him a specific color to recognize him
robot_0.attach_routine(foraging_drive)
robot_0.color = "black"

As for a behavior, a routine is attached to an robot at a given frequency. However, contrary to behaviors, routines are not directly involved in motor control and therefore can be run at a lower frequency (1Hz above). The code of the `foraging_drive` function above will be executed each second (1Hz frequency), consequently modulating the robot energy level stored in `robot.energy_level` according to how good the robot is at catching spheres.

Before starting a routine, we need to define an initial level of energy. Let's do it on `robot1`:

## Comment : 

- the start object apparition works well but often you need to call it a lot of times
- bc it either spawns the objects too slowly or bc too fast
- and then stops when the max objects have been placed in the scene

--> Could find another way to make this feature better for users (e.g not stopping the fn even when the max objects have been reached, and have to call stop objects apparition so it stops) 

This is mandatory because `energy_level` is incremented or decremented in the `foraging_drive` definition. If not initialized, it will throw an error.

Let's now effectively start this routine:

And check the evolution of the energy level in `robot_0`:

In [43]:
print(robot_0.energy_level)

59.18000000000016


In [44]:
print(robot_0.energy_level)

59.05000000000019


Let the robot behave in the environment and re-execute the cell above to track the evolution of the energy level.

**Q3:** Modify the `foraging` behavior so that it is weighted according to the energy level of the robot: the lower the energy level, the higher `foraging` is weighted. However, as seen in Q1 above, the robot might still be attracted to spheres even when the `foraging` weight is null. Solve this issue by adding another behavior that drives the robot away from sphere according to the energy level: the higher the energy level, the more the robot is repulsed from spheres.

## Sensing other robot's attributes

It can be useful to allow a robot to sense attributes from other robots. For example, one might want to define different species of robot (cats and mouses for example) so that how a robot interact with another depends on their respective species. To achieve this, it is **not** recommended to modify the robot labels directly in V-REP (as a general rule, never modify the object labels in the VREP scene). Instead, each robot can be set with a specific attribute, e.g `robot.species = "cat"` and that attribute can be sensed by other robots whenever it is detected by a proximeter. Let's try it with three robots:

In [45]:
# close_session(simulator)
# simulator, robot1, robot2, robot3 = open_session(n_robots=3)

where we define a `species` attribute for each robot:

In [46]:
robot_0.species = "cat"
robot_0.diameter = 12.
robot_0.color = "red"
robot_0.can_eat = False

robot_1.species = robot_2.species = "mouse"
robot_1.diameter = robot_2.diameter = 7.
robot_1.color = robot_2.color = "cyan"

So we have one cat and two mouses. Place the robots so that `robot3` (mouse) is sensing `robot1` (cat) on its left proximeter and `robot2` (mouse) on its right proximeter. One can access attributes of sensed robots by first executing:

This cell above calls the `prox_activations` function of `robot3` with an additional argument, `return_robots`, which is set to `True`. When called like this, `prox_activations` will return the left and right proximeter activations as usual, as well as the references to the robots that are sensed by the proximeters, if any. Then we can access a specific attribute of those robots by executing:

To return sensed entities by left and right proximeters, we can use the `sensed_entities` function of the agents. This returns the left and right entities if they are sensed, and `None` otherwise. 

In [67]:
ent_l, ent_r = robot_0.sensed_entities()

We can check what are the entities detected by a robot by running the following cell:

In [69]:
# Give a specific color to robot_0 to recognize him
robot_0.color = "black"
# Sense entities on the left and right of robot_0
ent_l, ent_r = robot_0.sensed_entities()

# Print the infos of the sensed entities
print("Left entity:")
if ent_l:
    ent_l.print_infos()
else:
    print("No entity sensed")

print("\nRight entity:")
if ent_r:
    ent_r.print_infos()
else:
    print("No entity sensed")

Left entity:
No entity sensed

Right entity:
Entity Overview:
--------------------
Type: OBJECT
Subtype: resources
Idx: 9
Exists: True
Position: x=172.37, y=274.10



You can also directly sense specific attributes of these sensed entities with the `sense_attributes` function. Make sure to specify the `sensed_attribute` you want to sense (e.g 'diameter', 'species', etc.). You can also specify the `default_value` value to return if the attribute is not found (otherwise it is set to None). The function will return the given attribute of the left and right sensed entities if it exists (it can be either if the entity doesn't exist, or because the entity doesn't possess the specific attribute), and the default value otherwise.

For example, each entity has a diameter, so if you see a `None` value, it means that the entity isn't sensed by the agent.

In [ ]:
l_attr, r_attr = robot_0.sense_attributes(sensed_attribute="diameter",)
print(f"Left: {l_attr} Right: {r_attr}")

Left: None Right: 5.0


For this other case, we only defined species for the robots. We set the default value to 'unknown', so if an entity isn't detected, or doesn't have the attribute, it will return 'unknown'.

In [72]:
l_attr, r_attr = robot_0.sense_attributes(sensed_attribute="species", default_value="unknown")
print(f"Left: {l_attr} Right: {r_attr}")

Left: unknown Right: unknown


The `sensed_robot_attributes` function takes 4 arguments: the robot sensed by the left proximeter (called `robot_left` in the example above), the one sensed by the right proximeter (called `robot_right` above), the name of the attribute we are interested in (here `"species"`) and the default value to return if a proximeter is not sensing any robot (here `none`).

Move the robots in the scene and check how the attribute sensing is changing accordingly by re-executing the two above cells. Then try with other attributes, for example by setting a `age` attribute to the robots and sensing it from the proximeters.

Let's now extend the `fear` behavior so that mouses are only afraid by the cat but not by the other mouse:

In [73]:
def fear_cat(robot):
    (left, right) = robot.sensors(sensed_entities=["robots"])
    left_species, right_species = robot.sense_attributes(sensed_attribute="species", default_value ="none")
    left_activation = left if left_species == "cat" else 0
    right_activation = right if right_species == "cat" else 0
    return left_activation, right_activation

And run this behavior together with the `obstacle_avoidance` one on `robot3`:

In [74]:
robot_2.detach_all_behaviors()
robot_2.attach_behavior(obstacle_avoidance)
robot_2.attach_behavior(fear_cat)
robot_2.start_all_behaviors()

In [76]:
robot_2.print_behaviors()
robot_2.color = "silver"

Available behaviors: ['obstacle_avoidance', 'fear_cat'], Active behaviors: ['obstacle_avoidance', 'fear_cat']


Check in the simulator that `robot3` (mouse) is now avoiding `robot1` (cat) but not `robot2` (mouse). We can modify the `species` attribute of robots on the fly. Let's swap the species of `robot1` and `robot2`:

In [77]:
robot_0.species = "mouse"
robot_1.species = "cat"

Check in the simulator that `robot3` (mouse) is now avoiding `robot2` (cat) but not `robot1` (mouse).

**Q4:** Use the new functionalities we have seen in this session to design the following system:
Two robots are equipped with behaviors to avoid obstacle and catch spheres, as well as being attracted or repulsed by the other robot. Attraction and repulsion depend on the energy level of the other robot: the higher this level the more attraction, the lower this level the more repulsion. 

In [78]:
for ag in controller.agents:
    ag.detach_all_behaviors(stop_motors=True)

In [79]:
# Your code here


**Q5**: Modify the previous simulation such that attraction and repulsion depend on how much a robot has been close to others in the recent past. To do so, define a `social_drive` routine that modulates the value of a `social_need` attribute in each robot. This social need continuously increases when other robots are far and decreases when a robot comes closer to its conspecifics.

In [ ]:
stop_server_and_interface()
controller.stop()

Now that you finished this session, you can either jump to :

- [session 5](session_5_bonus.ipynb) : Bonus session that explains how to use routines to a simple Eco-Evolutionary simulation
- [session 6](session_6_logging.ipynb) : How to log and plot data about the simulation


## About the mini-project

The aim of the mini project is to show that you have understood all the concepts we have seen during these three practical sessions and that you can integrate them in a single demo. You can of course modify the V-REP scene at your will, e.g adding cups or trees. Don't forget to save you modified V-REP scene (`File -> Save scene as` in V-REP). You can start your project in a new Jupyter Notebook (`File -> New notebook -> Python3`) in the menu bar at the top of this notebook). Use both text cells (explaining what you are doing) and code cells (with the corresponding code to execute).

Therefore, your project will consists of two files: a V-REP scene (extension `.ttt`) a Jupyter Notebook (extension `.ipynb`). Don't forget to save them, e.g. on a USB stick, before logging out from the UPF computers. See [this slide](https://docs.google.com/presentation/d/1FlAUyvNynYU4mDBE2o20pf2Bn5PC_9Id3SK8sCMfr-Q/edit#slide=id.g34900c6ead_0_0) for more information on the mini-project.